In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt

In [0]:
class LayerBlock(nn.Module):
    def __init__(self,in_dim,out_dim,down=False):
        super(LayerBlock,self).__init__()
  
        self.bn1 = nn.BatchNorm2d(out_dim)  
        self.bn2 = nn.BatchNorm2d(out_dim)
        self.bn3 = nn.BatchNorm2d(out_dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_dim,out_dim,3,1,1)
        self.down = down
        self.in_dim = in_dim
        self.out_dim =out_dim
        self.match_size=nn.Conv2d(self.in_dim,self.out_dim,1)

        if self.down:
            self.conv1 = nn.Conv2d(in_dim,out_dim,3,2,1)
            self.layer = nn.Sequential(
                self.conv1,
                self.bn1,
                self.relu,
                self.conv2,
                self.bn2

            )
            self.downsampler = nn.Conv2d(in_dim,out_dim,1,2)
        else:
            self.conv1 = nn.Conv2d(in_dim,out_dim,3,1,1)
            self.layer = nn.Sequential(
                self.conv1,
                self.bn1,
                self.relu,
                self.conv2,
                self.bn2

            )
    
    def forward(self,x):
        if self.down:
            down_x = self.downsampler(x)
            down_x = self.bn3(down_x)
            out = self.layer(x)
            out = out + down_x
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.match_size(x)
                x = self.bn3(x)

            out = out+x
        out = self.relu(out)
        return out






class MyResNet(nn.Module):
    def __init__(self):
        super(MyResNet,self).__init__()
        self.conv1 = nn.Conv2d(3,64,7,2,3)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(3,2)
        self.layer1 = nn.Sequential(
            LayerBlock(64,64),
            LayerBlock(64,64),)
        self.layer2 = nn.Sequential(
            LayerBlock(64,128,True),
            LayerBlock(128,128)
        )
        self.layer3 = nn.Sequential(
            LayerBlock(128,256,True),
            LayerBlock(256,256)
        )
        self.layer4 = nn.Sequential(
            LayerBlock(256,512,True),
            LayerBlock(512,512)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dense = nn.Linear(512,10)

    def forward(self,x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size()[0],-1)
        x = self.dense(x)

        return x


In [34]:
transform_train = transforms.Compose([
                               
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomCrop(size=(32,32),padding=4),
                                transforms.ToTensor(),
                                transforms.Normalize( (0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
                                ])
transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
train_set = torchvision.datasets.CIFAR10(root='./',train=True,download=True,transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./',train=False,download=True,transform=transform_test)


Files already downloaded and verified
Files already downloaded and verified


In [0]:
train_set, val_set = torch.utils.data.random_split(train_set,[int(len(train_set)*0.9),int(len(train_set)*0.1)])

In [0]:
batch_size = 128 

train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True,drop_last=True)
val_loader =torch.utils.data.DataLoader(val_set,batch_size = batch_size,shuffle=True,drop_last = True)
test_loader =torch.utils.data.DataLoader(test_set,batch_size=batch_size,drop_last=True)

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = MyResNet().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),weight_decay=0.0001,lr=0.1,momentum=0.9)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer,[80,120],gamma=0.1)

cuda:0


In [39]:
# best_val_acc = 0
epochs = 160
# patience_epoch = 100
for i in range(epochs):
    model.train()
    correct = 0
    total = 0
    for j,[img,label] in enumerate(train_loader):
        x = img.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        _,output_index = torch.max(output,1)
        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        total += y.size()[0]
        correct += (output_index == y).sum().float()
     
    
    
    
    
    lr_scheduler.step() 
    # if best_val_acc <= 100*val_correct/val_total:
    #     best_val_acc = 100*val_correct/val_total
    #     patience = patience_epoch
    # elif patience == 0:
    #     print('No improvement on val_acc : Stop training')
    #     break
    # else:
    #     patience -= 1
    if i % (epochs/10) == 0:
        with torch.no_grad():
            val_correct = 0
            val_total = 0
            for k,[val_img,val_label] in enumerate(val_loader):
                val_x = val_img.to(device)
                val_y = val_label.to(device)
                val_output = model.forward(val_x)
                _,val_output_index = torch.max(val_output,1)
                val_loss = loss_func(val_output,val_y)
                val_total += val_y.size()[0]
                val_correct += (val_output_index==val_y).sum().float()
        print('EPOCHS {:>3d}/{} | train_loss : {:.4f} | train_acc : {:.4f} | val_loss : {:.4f} | val_acc : {:.4f}'\
            .format(i+1,epochs,loss,correct*100/total,val_loss,100*val_correct/val_total))  
# print('Best Validation Accuracy : {:.4f}'.format(best_val_acc))

EPOCHS   1/160 | train_loss : 1.5901 | train_acc : 30.9095 | val_loss : 1.6264 | val_acc : 37.8005
EPOCHS  17/160 | train_loss : 0.6255 | train_acc : 78.0315 | val_loss : 0.6371 | val_acc : 77.6242
EPOCHS  33/160 | train_loss : 0.7292 | train_acc : 83.3200 | val_loss : 0.6141 | val_acc : 80.2684
EPOCHS  49/160 | train_loss : 0.4444 | train_acc : 85.0294 | val_loss : 0.6500 | val_acc : 80.6891
EPOCHS  65/160 | train_loss : 0.5930 | train_acc : 86.3938 | val_loss : 0.4417 | val_acc : 80.8093
EPOCHS  81/160 | train_loss : 0.2003 | train_acc : 91.3328 | val_loss : 0.4007 | val_acc : 84.8958
EPOCHS  97/160 | train_loss : 0.0648 | train_acc : 95.8845 | val_loss : 0.6385 | val_acc : 86.2380
EPOCHS 113/160 | train_loss : 0.1873 | train_acc : 96.8327 | val_loss : 0.5304 | val_acc : 85.9175
EPOCHS 129/160 | train_loss : 0.0723 | train_acc : 98.0547 | val_loss : 0.5539 | val_acc : 86.4383
EPOCHS 145/160 | train_loss : 0.0396 | train_acc : 98.2261 | val_loss : 0.3567 | val_acc : 86.5986


In [40]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    
    for image,label in test_loader:
        x = image.to(device)
        y = label.to(device)

        output = model.forward(x)
        _,output_index = torch.max(output,1)
        total += label.size(0)
        correct += (output_index == y).sum().float()

    print('Accuracy of Testset : {:.4f}'.format(100*correct/total))

Accuracy of Testset : 87.4199
